# BIOMASS TDS

## Functions and classes

In [2]:
import json
from dataclasses import dataclass
from typing import Optional

import numpy as np
import pandas as pd
from biotm import const, tm
from procsim.core.version import __version__ as procsim_version


# Definitions
ORBITS_PER_RC = 44  # 44 orbits in 3 days
ORBIT_DURATION = 86400 * 3 / 44
ORBITS_PER_SRM = int(ORBITS_PER_RC / 3 * 9)
SWATHS = 3
TOM_RC_PER_SWATH = 7
INT_RC_PER_SWATH = 3
TOM_ORBITS_PER_SWATH = ORBITS_PER_RC * TOM_RC_PER_SWATH
INT_ORBITS_PER_SWATH = ORBITS_PER_RC * INT_RC_PER_SWATH
TOM_ORBITS_PER_MC = TOM_ORBITS_PER_SWATH * SWATHS + ORBITS_PER_SRM
INT_ORBITS_PER_MC = INT_ORBITS_PER_SWATH * SWATHS + ORBITS_PER_SRM
TOM_ORBITS_PER_GC = TOM_ORBITS_PER_MC * 6 + TOM_ORBITS_PER_SWATH + ORBITS_PER_SRM
INT_ORBITS_PER_GC = INT_ORBITS_PER_MC * 6 + INT_ORBITS_PER_SWATH + ORBITS_PER_SRM
SAR_ISPS_PER_SEC = int(np.ceil(1 / 340e-6))  # Assumption: SAR PRI = 340 us
ANC_ISPS_PER_SEC = 1
BASELINE = 1
SLICE_MIN_DURATION = 28
FRAME_MIN_DURATION = 7


@dataclass
class AbsoluteOrbit:
    
    orbit: int
    mission_phase: tm.MissionPhase
    start: Optional[pd.Timestamp] = None
    stop: Optional[pd.Timestamp] = None
    
    def orbits_per_mc(self) -> int:
        if self.mission_phase == tm.MissionPhase.TOM:
            result = TOM_ORBITS_PER_MC
        elif self.mission_phase == tm.MissionPhase.INT:
            result = INT_ORBITS_PER_MC
        else:
            raise ValueError("phase must be tm.MissionPhase.TOM or tm.MissionPhase.INT")
        return result

    def orbits_per_swath(self) -> int:
        if self.mission_phase == tm.MissionPhase.TOM:
            result = TOM_ORBITS_PER_SWATH
        elif self.mission_phase == tm.MissionPhase.INT:
            result = INT_ORBITS_PER_SWATH
        else:
            raise ValueError("phase must be tm.MissionPhase.TOM or tm.MissionPhase.INT")
        return result
    
    def orbits_per_gc(self) -> int:
        if self.mission_phase == tm.MissionPhase.TOM:
            result = TOM_ORBITS_PER_GC
        elif self.mission_phase == tm.MissionPhase.INT:
            result = INT_ORBITS_PER_GC
        else:
            raise ValueError("phase must be tm.MissionPhase.TOM or tm.MissionPhase.INT")
        return result
    
    def verify_orbit(self):
        if self.orbit < 1:
            raise ValueError("orbit must be > 0")
            
    def gcid(self) -> int:
        self.verify_orbit()
        return int(np.ceil(self.orbit / self.orbits_per_gc()))

    def mcid(self) -> int:
        self.verify_orbit()
        orbit = self.orbit - (self.gcid() - 1) * self.orbits_per_gc()
        result = int(np.ceil(orbit / self.orbits_per_mc()))
        if result > 7:
            raise ValueError("computed MCID is greater than 7")
        return result
    
    def swathid(self) -> int:
        self.verify_orbit()
        orbit = self.orbit - (self.gcid() - 1) * self.orbits_per_gc() - (self.mcid() - 1) * self.orbits_per_mc()
        swaths = SWATHS if self.mcid() < 7 else 1
        if orbit < self.orbits_per_swath() * swaths:
            result = int(np.ceil(orbit / self.orbits_per_swath()))
        else:
            # SRM
            result = 0
        return result
    
    def rcid(self) -> int:
        self.verify_orbit()
        swathid = self.swathid()
        if swathid == 0:
            # SRM
            result = 0
        else:
            orbit = self.orbit - (self.gcid() - 1) * self.orbits_per_gc() - (self.mcid() - 1) * self.orbits_per_mc() - \
                (swathid - 1) * self.orbits_per_swath()
            result = int(np.ceil(orbit / ORBITS_PER_RC))
        return result
    
    def track(self) -> int:
        self.verify_orbit()
        swathid = self.swathid()
        if swathid == 0:
            # SRM
            result = 0
        else:
            result = self.orbit - (self.gcid() - 1) * self.orbits_per_gc() - (self.mcid() - 1) * self.orbits_per_mc() - \
                (swathid - 1) * self.orbits_per_swath() - (self.rcid() - 1) * ORBITS_PER_RC
        return result
    
    def to_interval(self) -> pd.Interval:
        if None in (self.start, self.stop):
            raise TypeError("orbit's start and stop times must be not None")
        return pd.Interval(self.start, self.stop, closed='both')


@dataclass
class Grid:
    
    orbit_start: pd.Timestamp
    orbit_stop: pd.Timestamp
    duration: float
    initial_overlap: float
    final_overlap: float
    num_intervals: int
        
    def interval(self, n: int) -> pd.Interval:
        if n < 1 or n > self.num_intervals:
            raise ValueError(f'interval number must be included in [0, {self.num_intervals}]')
        left = self.orbit_start + pd.to_timedelta((n - 1) * self.duration - self.initial_overlap, 's')
        if n == self.num_intervals:
            right = self.orbit_stop + pd.to_timedelta(self.final_overlap, 's')
        else:
            right = self.orbit_start + pd.to_timedelta(n * self.duration + self.final_overlap, 's')
        return pd.Interval(left, right, closed='both')
    
    def intervals(self, restriction: Optional[pd.Interval] = None,
                  min_duration: Optional[float] = None) -> list[tuple[int, pd.Interval]]:
        result = []
        for n in range(self.num_intervals):
            i = self.interval(n + 1)
            if restriction is None:
                result.append((n + 1, i))
            else:
                if i.overlaps(restriction):
                    start = max(restriction.left, i.left)
                    stop = min(restriction.right, i.right)
                    result.append((n + 1, pd.Interval(start, stop, closed='both')))
        if min_duration is not None:
            _result = []
            for k in range(len(result)):
                if result[k][1].length.total_seconds() >= min_duration:
                    start = result[k][1].left
                    stop = result[k][1].right
                    if k - 1 >= 0 and result[k - 1][1].length.total_seconds() < min_duration:
                        start = result[k - 1][1].left
                    if k + 1 < len(result) and result[k + 1][1].length.total_seconds() < min_duration:
                        stop = result[k + 1][1].right
                    _result.append((result[k][0], pd.Interval(start, stop, closed='both')))
            result = _result
        return result

    
@dataclass
class SliceGrid(Grid):

    orbit_start: pd.Timestamp
    orbit_stop: pd.Timestamp
    duration: float = const.SLICE_GRID_DURATION
    initial_overlap: float = const.SLICE_INITIAL_OVERLAP
    final_overlap: float = const.SLICE_FINAL_OVERLAP
    num_intervals: int = const.NUM_SLICES

        
@dataclass
class FrameGrid(Grid):

    orbit_start: pd.Timestamp
    orbit_stop: pd.Timestamp
    duration: float = const.FRAME_GRID_DURATION
    initial_overlap: float = const.FRAME_INITIAL_OVERLAP
    final_overlap: float = const.FRAME_FINAL_OVERLAP
    num_intervals: int = const.NUM_FRAMES

        
def get_orbits(simulation: pd.DataFrame, idx: int) -> list[pd.Interval]:
    """Get list of orbit intervals overlapping acuisition identified by `idx`."""
    return [
        pd.Interval(simulation.loc[idx]['Anx'], simulation.loc[idx]['NextAnx'], closed='both'),
        pd.Interval(
            simulation[simulation['Orbit'] == simulation.loc[idx]['Orbit'] + 1]['Anx'].min(),
            simulation[simulation['Orbit'] == simulation.loc[idx]['Orbit'] + 1]['NextAnx'].min(),
            closed='both'
        )
    ]


def get_slices(orbits: list[pd.Interval], interval: pd.Interval, threshold=int) -> pd.DataFrame:
    """Get list of slices having overlap with `interval`."""
    slices = []
    for orbit in orbits:
        sg = SliceGrid(orbit.left, orbit.right)
        slices.extend(sg.intervals(interval, threshold))  
    return pd.DataFrame(
        [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
         for s in slices]
    )


def get_frames(orbits: list[pd.Interval], interval: pd.Interval, threshold=int) -> pd.DataFrame:
    """Get list of frames having overlap with `interval`."""
    frames = []
    for orbit in orbits:
        fg = FrameGrid(orbit.left, orbit.right)
        frames.extend(fg.intervals(interval, threshold))
    return pd.DataFrame(
        [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
         for f in frames]
    )


def raw_generation_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str) -> dict:
    if mode == 'MM':
        pids = [25, 26]
    elif mode == 'RO':
        pids = [27, 28]
    elif mode == 'EC':
        pids = [35, 36]
    else:
        raise ValueError(f"unknown mode: {mode}")
    
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    plt_data_validity = pd.Interval(
        data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
        data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
        closed='both'
    )
    acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
    return {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']}",
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': 'SV',
        'baseline': 1,
        'outputs': [
            {
                'type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in pids
        ]
    }


def l0pfs1_processing_scenarios(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> list[dict]:
    pids = [22, 23, 24]
    if mode == 'MM':
        pids += [25, 26]
    elif mode == 'RO':
        pids += [27, 28]
    elif mode == 'EC':
        pids += [35, 36]
    else:
        raise ValueError(f"unknown mode: {mode}")

    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = [
        {
            'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L0PFS1 - RAW_{pid:03d}_10",
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{pid}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{pid}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'type': f'RAWS{pid:03d}_10',
                    'metadata_source': f'.*RAW_{pid:03d}_10_.*'
                }
            ]
        }
        for pid in pids
    ]
    return result


def l0pfs2_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    if mode == 'MM':
        outputs = [
            {
                'type': f"{simulation.loc[idx]['SensorMode'][-2:]}_RAW__0S",
                'metadata_source': '.*RAWS025_10_.*'
            },
            {
                'type': f"{simulation.loc[idx]['SensorMode'][-2:]}_RAW__0S",
                'metadata_source': '.*RAWS025_10_.*'
            }            
        ]
    elif mode == 'RO':
        outputs = [
            {
                'type': "RO_RAW__0S",
                'metadata_source': '.*RAWS027_10_.*'
            }
        ]
    elif mode == 'EC':
            {
                'type': "EC_RAWP_0S",
                'metadata_source': '.*RAWS035_10_.*'
            },
            {
                'type': "EC_RAWP_0M",
                'metadata_source': '.*RAWS035_10_.*'
            }            
    else:
        raise ValueError(f"unknown mode: {mode}")
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L0PFS2",
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': simulation.loc[idx]['SensorMode'][-2:],
                'operational_mode': mode
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': outputs
    }
    return result


def l0pfs3_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    if mode == 'MM':
        swath_id = simulation.loc[idx]['SensorMode'][-2:]
    elif mode == 'EC':
        swath_id = 'RO'
    else:
        raise ValueError(f"unknown mode: {mode}")
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L0PFS3",
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': mode
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',                
            }
        ]
    }
    return result


def l0pfs4_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L0PFS4",
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': simulation.loc[idx]['SensorMode'][-2:],
                'operational_mode': mode
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(ANC_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
    return result


def l0pfs5_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L0PFS5",
        'file_name': 'l0pfs5.sh',
        'processor_name': 'L0PFS5',
        'processor_version': '01.00',
        'task_name': 'L0PFS5',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': 'EC',
                'operational_mode': 'EC'
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'EC_RAW__0S',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
            }
        ]
    }
    return result


def oapf_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - OAPF",
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': mode
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
    return result


%reload_ext watermark
%watermark -iv

numpy : 1.20.2
pandas: 1.2.5
biotm : 1.0a0
json  : 2.0.9



## TOM_TDS

### Load data from SaVoir simulation

In [3]:
def next_anx(row):
    next_orbit = row['Orbit'] + 1
    next_anx = row['Anx'] + pd.to_timedelta(ORBIT_DURATION, 's')
    r = simulation.query("Orbit == @next_orbit")
    if r.shape[0] > 0:
        next_anx = r['Anx'].min()
    return next_anx

simulation = pd.read_excel('Export_TOM_Full_GC_S1_S2_S3_RO_S1SRM.xlsx')

# add DTID
simulation['DTID'] = [int(tm.DataTakeID(row.Orbit, row.TAnx).tobytes().hex(), 16) for row in simulation.itertuples()]
# add mcid, rcid, track, ext anx
orbits = [AbsoluteOrbit(n, tm.MissionPhase.INT) for n in simulation['Orbit']]
simulation['gcid'] = [o.gcid() for o in orbits]
simulation['mcid'] = [o.mcid() for o in orbits]
simulation['rcid'] = [o.rcid() for o in orbits]
simulation['track'] = [o.track() for o in orbits]
simulation['NextAnx'] = simulation.apply(next_anx, axis=1)

simulation.sort_values(by=['Start'], inplace=True)

simulation.columns

Index(['Start', 'End', 'Duration', 'Satellite', 'SensorMode', 'Width',
       'Length', 'SwathArea', 'Pass', 'NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon',
       'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'Orbit', 'Anx', 'AnxLon',
       'TAnx', 'DTID', 'gcid', 'mcid', 'rcid', 'track', 'NextAnx'],
      dtype='object')

### MM scenario

#### Data Take selection

Select Data Take across ANX

In [4]:
simulation.query("End > NextAnx and SensorMode != 'Receive Only'")[['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'SensorMode']].head(10)

,Anx,Start,End,NextAnx,Duration,SensorMode
6,2017-01-01 07:38:12.217,2017-01-01 09:09:51.641,2017-01-01 09:18:02.670,2017-01-01 09:16:23.162,491.029120,Stripmap S1
16,2017-01-01 09:16:23.162,2017-01-01 10:50:05.171,2017-01-01 10:56:37.753,2017-01-01 10:54:34.108,392.582341,Stripmap S1
51,2017-01-01 20:43:39.780,2017-01-01 22:21:04.727,2017-01-01 22:22:44.587,2017-01-01 22:21:50.725,99.859843,Stripmap S1
64,2017-01-02 01:38:12.615,2017-01-02 03:15:11.450,2017-01-02 03:27:17.189,2017-01-02 03:16:23.561,725.739219,Stripmap S1
66,2017-01-02 03:16:23.561,2017-01-02 04:45:17.522,2017-01-02 04:58:17.721,2017-01-02 04:54:34.506,780.199136,Stripmap S1
77,2017-01-02 08:10:56.397,2017-01-02 09:40:05.106,2017-01-02 09:52:17.280,2017-01-02 09:49:07.342,732.174332,Stripmap S1
112,2017-01-02 19:38:13.014,2017-01-02 21:15:25.695,2017-01-02 21:17:08.071,2017-01-02 21:16:23.959,102.376548,Stripmap S1
119,2017-01-02 21:16:23.959,2017-01-02 22:52:34.752,2017-01-02 22:56:26.232,2017-01-02 22:54:34.905,231.479987,Stripmap S1
130,2017-01-03 02:10:56.796,2017-01-03 03:42:33.956,2017-01-03 03:58:21.488,2017-01-03 03:49:07.741,947.531690,Stripmap S1
132,2017-01-03 03:49:07.741,2017-01-03 05:26:48.577,2017-01-03 05:27:20.577,2017-01-03 05:27:18.686,32.000000,Stripmap S1


ID of selected acquisition: 51

In [5]:
acq_idx = 51

In [6]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-01 22:21:04.727000', '2017-01-01 22:22:44.587000', closed='both')

In [7]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

87764

#### Swath

In [8]:
# get swath id from SensorMode name
swath_id = simulation.loc[acq_idx]['SensorMode'][-2:]
swath_id

'S1'

In [9]:
# crosscheck with orbit number
AbsoluteOrbit(simulation.loc[acq_idx]['Orbit'], tm.MissionPhase.INT).swathid()

1

#### Mission phase, global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [10]:
columns = ['Anx', 'Start', 'End', 'NextAnx', 'gcid', 'mcid', 'rcid', 'Orbit', 'track']
simulation.loc[acq_idx: acq_idx + 1][columns]

,Anx,Start,End,NextAnx,gcid,mcid,rcid,Orbit,track
51,2017-01-01 20:43:39.780,2017-01-01 22:21:04.727,2017-01-01 22:22:44.587,2017-01-01 22:21:50.725,1,1,1,10,10
52,2017-01-01 22:21:50.725,2017-01-01 22:25:11.726,2017-01-01 22:26:26.213,2017-01-02 00:00:01.670,1,1,1,11,11


In [11]:
mission_phase = 'TOMOGRAPHIC'
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [12]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [13]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-01 22:52:44.587000')

#### Affected Orbits

Orbits overlapping data take.

In [14]:
orbits = get_orbits(simulation, acq_idx)
orbits

[Interval('2017-01-01 20:43:39.780000', '2017-01-01 22:21:50.725000', closed='both'),
 Interval('2017-01-01 22:21:50.725000', '2017-01-02 00:00:01.670000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [15]:
slices = get_slices(orbits, data_take, SLICE_MIN_DURATION)
slices

,n,start,stop,duration
0,62,2017-01-01 22:21:04.727,2017-01-01 22:21:57.725,52.998
1,1,2017-01-01 22:21:45.725,2017-01-01 22:22:44.587,58.862


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [16]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = get_slices(orbits, plt_data_validity, SLICE_MIN_DURATION)
plt_slices

,n,start,stop,duration
0,62,2017-01-01 22:20:48.727,2017-01-01 22:21:57.725,68.998
1,1,2017-01-01 22:21:45.725,2017-01-01 22:22:44.587,58.862


#### Level 1 frames

Frame intervals overlapping data take:

In [17]:
frames = get_frames(orbits, data_take, FRAME_MIN_DURATION)
frames

,n,start,stop,duration
0,308,2017-01-01 22:21:04.727000000,2017-01-01 22:21:14.753756784,10.026756
1,309,2017-01-01 22:21:12.753756784,2017-01-01 22:21:33.756918332,21.003161
2,310,2017-01-01 22:21:31.756918332,2017-01-01 22:21:52.725000000,20.968081
3,1,2017-01-01 22:21:50.725000000,2017-01-01 22:22:11.728161548,21.003161
4,2,2017-01-01 22:22:09.728161548,2017-01-01 22:22:30.731323096,21.003161
5,3,2017-01-01 22:22:28.731323096,2017-01-01 22:22:44.587000000,15.855676


#### Additional data takes for stack

Selected slice and frame:
- slice: 62
- frame: 310

Select additional data takes having the same global coverage id, major cycle id, swath (sensor mode) and track:

In [18]:
sensor_mode = simulation.loc[acq_idx]['SensorMode']
track = simulation.loc[acq_idx]['track']
result = simulation.query("gcid == @gcid and mcid == @mcid and SensorMode == @sensor_mode and track == @track")
# discard orbit of previously selected data take
orbit = simulation.loc[acq_idx]['Orbit']
result = result.query("Orbit != @orbit")
result

,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,NW_Lat,...,Orbit,Anx,AnxLon,TAnx,DTID,gcid,mcid,rcid,track,NextAnx
248,2017-01-04 20:53:21.854,2017-01-04 21:05:21.619,719.764550,BIOMASS,Stripmap S1,54.684693,4967.733345,271387.917982,ASCENDING,74.999049,...,54,2017-01-04 20:43:41.374,139.082902,580.480484,442948,1,1,2,10,2017-01-04 22:21:52.319
250,2017-01-04 21:32:03.705,2017-01-04 21:43:12.521,668.815337,BIOMASS,Stripmap S1,54.398844,4576.589684,250156.067481,DESCENDING,1.877719,...,54,2017-01-04 20:43:41.374,139.082902,2902.331558,445270,1,1,2,10,2017-01-04 22:21:52.319
251,2017-01-04 21:49:05.892,2017-01-04 21:55:00.163,354.271211,BIOMASS,Stripmap S1,56.222801,2405.168665,134927.914668,DESCENDING,-59.882895,...,54,2017-01-04 20:43:41.374,139.082903,3924.517712,446292,1,1,2,10,2017-01-04 22:21:52.319
252,2017-01-04 22:12:32.713,2017-01-04 22:14:21.115,108.402255,BIOMASS,Stripmap S1,54.872305,740.261241,40703.771640,ASCENDING,-28.000025,...,54,2017-01-04 20:43:41.374,139.082903,5331.338837,447699,1,1,2,10,2017-01-04 22:21:52.319
253,2017-01-04 22:19:32.247,2017-01-04 22:20:04.247,32.000000,BIOMASS,Stripmap S1,54.050728,219.419615,11945.923227,ASCENDING,-7.155168,...,54,2017-01-04 20:43:41.374,139.082902,5750.872933,448118,1,1,2,10,2017-01-04 22:21:52.319
254,2017-01-04 22:21:06.291,2017-01-04 22:22:46.135,99.844118,BIOMASS,Stripmap S1,54.152229,685.542429,37209.155797,ASCENDING,2.698233,...,54,2017-01-04 20:43:41.374,139.082902,5844.916996,448212,1,1,2,10,2017-01-04 22:21:52.319
451,2017-01-07 20:53:23.399,2017-01-07 21:05:23.212,719.813049,BIOMASS,Stripmap S1,54.628558,4968.067740,271408.217437,ASCENDING,74.999054,...,98,2017-01-07 20:43:42.967,139.076258,580.431511,803396,1,1,3,10,2017-01-07 22:21:53.912
453,2017-01-07 21:32:05.266,2017-01-07 21:43:14.127,668.860385,BIOMASS,Stripmap S1,54.387678,4576.898158,250169.885640,DESCENDING,1.879676,...,98,2017-01-07 20:43:42.967,139.076258,2902.298893,805718,1,1,3,10,2017-01-07 22:21:53.912
454,2017-01-07 21:49:07.485,2017-01-07 21:55:01.760,354.274837,BIOMASS,Stripmap S1,56.222824,2405.192911,134935.122722,DESCENDING,-59.882915,...,98,2017-01-07 20:43:42.967,139.076258,3924.517579,806740,1,1,3,10,2017-01-07 22:21:53.912
455,2017-01-07 22:12:34.288,2017-01-07 22:14:22.709,108.420251,BIOMASS,Stripmap S1,54.885966,740.383945,40711.560118,ASCENDING,-28.000010,...,98,2017-01-07 20:43:42.967,139.076258,5331.320877,808147,1,1,3,10,2017-01-07 22:21:53.912


Select data takes having overlap with frame 310:

In [19]:
def overlap(data_take_start, data_take_stop, orbit_start, orbit_stop, frame_number):
    data_take = pd.Interval(data_take_start, data_take_stop, closed='both')
    frame_grid = FrameGrid(orbit_start, orbit_stop)
    return frame_grid.interval(frame_number).overlaps(data_take)

idx = result.apply(
    lambda r: overlap(r['Start'], r['End'], r['Anx'], r['NextAnx'], 310),
    axis=1
)
result = result[idx]
result[['Orbit', 'Anx', 'Start', 'End', 'NextAnx', 'Duration']]

,Orbit,Anx,Start,End,NextAnx,Duration
254,54,2017-01-04 20:43:41.374,2017-01-04 22:21:06.291,2017-01-04 22:22:46.135,2017-01-04 22:21:52.319,99.844118
457,98,2017-01-07 20:43:42.967,2017-01-07 22:21:07.852,2017-01-07 22:22:47.683,2017-01-07 22:21:53.912,99.831301
660,142,2017-01-10 20:43:44.560,2017-01-10 22:21:09.414,2017-01-10 22:22:49.233,2017-01-10 22:21:55.505,99.818513
863,186,2017-01-13 20:43:46.154,2017-01-13 22:21:10.977,2017-01-13 22:22:50.781,2017-01-13 22:21:57.100,99.803906
1066,230,2017-01-16 20:43:47.745,2017-01-16 22:21:12.539,2017-01-16 22:22:52.330,2017-01-16 22:21:58.690,99.791180
1269,274,2017-01-19 20:43:49.339,2017-01-19 22:21:14.102,2017-01-19 22:22:53.880,2017-01-19 22:22:00.284,99.778374


In [20]:
idxs = [acq_idx] + list(result.index.to_numpy())
idxs

[51, 254, 457, 660, 863, 1066, 1269]

#### Raw Data generation

Scenario generation

In [67]:
config = {
    'mission': 'biomass',
    'scenarios': [raw_generation_scenario('TOM_TDS', simulation, idx, 'MM') for idx in idxs]
}
with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [68]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l0pfs1_processing_scenarios('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(l0pfs2_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(l0pfs3_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(l0pfs4_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(oapf_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
with open('tom-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1a/b

In [69]:
config = {
    'mission': 'biomass',
    'scenarios': tds.l1_processing_scenarios(
        tds_name='TOM_TDS',
        data_take_num=1,
        data_take=data_take,
        footprint_polygon=footprint_polygon,
        slices=slices,
        baseline=1,
        orbits=orbits,
        swath_id=swath_id,
        mission_phase='TOMOGRAPHIC',
        dtid=dtid,
        gcid=gcid,
        mcid=mcid,
        rcid=rcid,
        track=track
    )
}
with open('tom-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

NameError: name 'tds' is not defined

### MM - Stack - additional data takes

In [30]:
for acq_idx in idxs:
    
    data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
    dtid = simulation.loc[acq_idx, 'DTID']
    
    mission_phase = 'TOMOGRAPHIC'
    gcid = simulation.loc[acq_idx]['gcid']
    mcid = simulation.loc[acq_idx]['mcid']
    rcid = simulation.loc[acq_idx]['rcid']
    track = simulation.loc[acq_idx]['track']
    
    dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()
    footprint_polygon = ' '.join(dt_polygon.astype(str))
    
    acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
    acquisition_station = 'SV'  # Svalbard

    orbits = [
        pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx'], closed='both'),
        pd.Interval(
            simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['Anx'].min(),
            simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['NextAnx'].min(),
            closed='both'
        )
    ]

    slices = []
    for orbit in orbits:
        sg = tds.SliceGrid(orbit.left, orbit.right)
        slices.extend(sg.intervals(data_take, 27))
    slices = pd.DataFrame(
        [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
         for s in slices]
    )

    plt_data_validity = pd.Interval(
        data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
        data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
        closed='both'
    )
    plt_slices = []
    for orbit in orbits:
        sg = tds.SliceGrid(orbit.left, orbit.right)
        plt_slices.extend(sg.intervals(plt_data_validity, 27))
    plt_slices = pd.DataFrame(
        [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
         for s in plt_slices]
    )

    frames = []
    for orbit in orbits:
        fg = tds.FrameGrid(orbit.left, orbit.right)
        frames.extend(fg.intervals(data_take, 7))
    frames = pd.DataFrame(
        [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
         for f in frames]
    )

    with open('tom-tds-raw-gen-scenarios.json') as fh:
        config=json.load(fh)
    config['scenarios'].extend(tds.raw_generation_scenarios('TOM_TDS', 1, data_take, 'MM', acquisition_date, acquisition_station, 1))
    with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
        json.dump(config, fh, indent=2)

    with open('tom-tds-l0-scenarios.json') as fh:
        config = json.load(fh)
    config['scenarios'].extend(
        tds.l0_processing_scenarios(
            tds_name='TOM_TDS',
            data_take_num=1,
            data_take=data_take,
            mode='MM',
            acquisition_date=acquisition_date,
            acquisition_station=acquisition_station,
            baseline=1,
            orbits=orbits,
            swath_id=swath_id,
            mission_phase='TOMOGRAPHIC',
            dtid=dtid,
            gcid=gcid,
            mcid=mcid,
            rcid=rcid,
            track=track
        )
    )
    with open('tom-tds-l0-scenarios.json', 'w') as fh:
        json.dump(config, fh, indent=2)

    with open('tom-tds-l1-scenarios.json') as fh:
        config = json.load(fh)
    config['scenarios'].extend(
        tds.l1_processing_scenarios(
            tds_name='TOM_TDS',
            data_take_num=1,
            data_take=data_take,
            footprint_polygon=footprint_polygon,
            slices=slices,
            baseline=1,
            orbits=orbits,
            swath_id=swath_id,
            mission_phase='TOMOGRAPHIC',
            dtid=dtid,
            gcid=gcid,
            mcid=mcid,
            rcid=rcid,
            track=track
        )
    )
    with open('tom-tds-l1-scenarios.json', 'w') as fh:
        json.dump(config, fh, indent=2)

In [24]:
frames.query('n == 310')

,n,start,stop,duration
2,310,2017-01-19 22:21:41.315918332,2017-01-19 22:22:02.284,20.968081


#### Level 1c / level 2a

In [26]:
simulation.loc[51]['gcid']
simulation.loc[51]['mcid']
simulation.loc[51]['track']

data_takes = []
dtids = []
footprint_polygons = []
frames = []
rcids = []
orbits = []
for acq_idx in [51] + list(idxs):
    data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
    data_takes.append(data_take)
    dtids.append(simulation.loc[acq_idx, 'DTID'])
    
    rcids.append(simulation.loc[acq_idx]['rcid'])
    
    orbits.append(pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx']))

    polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()
    footprint_polygons.append(' '.join(polygon.astype(str)))
    orbits = [
        pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx'], closed='both'),
        pd.Interval(
            simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['Anx'].min(),
            simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['NextAnx'].min(),
            closed='both'
        )
    ]
    _frames = []
    for orbit in orbits:
        fg = tds.FrameGrid(orbit.left, orbit.right)
        _frames.extend(fg.intervals(data_take, 7))
    _frames = pd.DataFrame(
        [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
         for f in _frames]
    )
    result = _frames.query('n == 310')
    frames.append(
        pd.Interval(result.iloc[0]['start'], result.iloc[0]['stop'], closed='both')
    )

config = {
    'mission': 'biomass',
    'scenarios': tds.stack_processing_scenario(
        tds_name='TOM_TDS',
        data_takes=data_takes,
        footprint_polygons=footprint_polygons,
        orbits=orbits,
        baseline=1,
        swath_id=swath_id,
        mission_phase='TOMOGRAPHIC',
        dtids=dtids,
        gcid=gcid,
        mcid=mcid,
        rcids=rcids,
        track=track,
        frames=frames
    )
}


NameError: name 'data_take' is not defined

In [ ]:
config['scenarios'].append(
    {
        'name': f'TOM_TDS - STACK',
        'file_name': 'stack_l2a.sh',
        'processor_name': 'STACK_L2A',
        'processor_version': '01.00',
        'task_name': 'STACK',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(gcid),
        'repeat_cycle_id': str(gcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'footprint_polygon': footprint_polygon,
        'outputs': [
            {
                'file_type': f'{swath_id}_STA__1S',
                'begin_position': frame.left.isoformat() + 'Z',
                'end_position': frame.right.isoformat() + 'Z'
            }
            for frame in frames
        ]
    }
)

config['scenarios'].append(
    {
        'name': f'TOM_TDS - L2A',
        'file_name': 'stack_l2a.sh',
        'processor_name': 'STACK_L2A',
        'processor_version': '01.00',
        'task_name': 'L2A',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(gcid),
        'repeat_cycle_id': str(gcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'footprint_polygon': footprint_polygon,
        'outputs': [
            {
                'file_type': 'FP_FD__L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_7.right.isoformat() + 'Z'
            },
            {
                'file_type': 'FP_FH__L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_7.right.isoformat() + 'Z'
            },
            {
                'file_type': 'FP_ACM_L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_7.right.isoformat() + 'Z'
            },
            {
                'file_type': 'FP_VBG_L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_7.right.isoformat() + 'Z'
            }
        ]
    }
)

In [ ]:
with open('int-tds-l1c-l2a-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### RO scenario

#### Data Take selection

Select Data Take across ANX

In [630]:
simulation.head()

,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,NW_Lat,...,Orbit,Anx,AnxLon,TAnx,DTID,gcid,mcid,rcid,track,NextAnx
0,2017-01-01 06:01:31.394,2017-01-01 06:03:44.504,133.110305,BIOMASS,Stripmap S1,53.961516,915.099690,49580.620718,ASCENDING,13.001162,...,1,2017-01-01 06:00:01.272,1.849826e-07,90.122152,8282,1,1,1,1,2017-01-01 07:38:12.217
1,2017-01-01 06:08:39.801,2017-01-01 06:36:30.245,1670.444243,BIOMASS,Receive Only,55.066325,11519.783482,630257.214895,ASCENDING,44.859358,...,1,2017-01-01 06:00:01.272,-2.086955e-08,518.528527,8710,1,1,1,1,2017-01-01 07:38:12.217
2,2017-01-01 06:52:49.307,2017-01-01 06:55:03.719,134.411778,BIOMASS,Stripmap S1,54.464226,919.977565,50248.148728,DESCENDING,-14.293988,...,1,2017-01-01 06:00:01.272,6.098976e-09,3168.035430,11360,1,1,1,1,2017-01-01 07:38:12.217
3,2017-01-01 07:49:10.607,2017-01-01 08:11:51.127,1360.520507,BIOMASS,Receive Only,54.972860,9388.170636,514126.843406,ASCENDING,54.860562,...,2,2017-01-01 07:38:12.217,-2.454561e+01,658.389814,17042,1,1,1,2,2017-01-01 09:16:23.162
4,2017-01-01 08:28:10.480,2017-01-01 08:29:15.351,64.870512,BIOMASS,Stripmap S1,53.780686,444.955991,24185.787974,DESCENDING,-3.962771,...,2,2017-01-01 07:38:12.217,-2.454561e+01,2998.263258,19382,1,1,1,2,2017-01-01 09:16:23.162


In [26]:
simulation.query("End > NextAnx and SensorMode == 'Receive Only'")[['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'SensorMode']].head(10)

,Anx,Start,End,NextAnx,Duration,SensorMode


ID of selected acquisition: 1

In [632]:
acq_idx = 1

In [633]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-01 06:08:39.801000', '2017-01-01 06:36:30.245000', closed='both')

In [634]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

8710

#### Swath

In [636]:
swath_id = 'RO'

#### Mission phase, global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [483]:
mission_phase = 'TOMOGRAFIC'
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [484]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [485]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-01 22:52:44.587000')

#### Affected Orbits

Orbits overlapping data take.

In [486]:
orbits = [
    pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx'], closed='both'),
    pd.Interval(
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['Anx'].min(),
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['NextAnx'].min(),
        closed='both'
    )
]
orbits

[Interval('2017-01-01 20:43:39.780000', '2017-01-01 22:21:50.725000', closed='both'),
 Interval('2017-01-01 22:21:50.725000', '2017-01-02 00:00:01.670000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [487]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,62,2017-01-01 22:21:04.727,2017-01-01 22:21:57.725,52.998
1,1,2017-01-01 22:21:45.725,2017-01-01 22:22:44.587,58.862


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [488]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,62,2017-01-01 22:20:48.727,2017-01-01 22:21:57.725,68.998
1,1,2017-01-01 22:21:45.725,2017-01-01 22:22:44.587,58.862


#### Level 1 frames

Frame intervals overlapping data take:

In [489]:
frames = []
for orbit in orbits:
    fg = Grid(orbit.left, orbit.right, const.FRAME_GRID_DURATION, const.FRAME_INITIAL_OVERLAP, const.FRAME_FINAL_OVERLAP, const.NUM_FRAMES)
    frames.extend(fg.intervals(data_take, 7))

frames = pd.DataFrame(
    [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
     for f in frames]
)
frames

,n,start,stop,duration
0,308,2017-01-01 22:21:04.727000000,2017-01-01 22:21:14.753756784,10.026756
1,309,2017-01-01 22:21:12.753756784,2017-01-01 22:21:33.756918332,21.003161
2,310,2017-01-01 22:21:31.756918332,2017-01-01 22:21:52.725000000,20.968081
3,1,2017-01-01 22:21:50.725000000,2017-01-01 22:22:11.728161548,21.003161
4,2,2017-01-01 22:22:09.728161548,2017-01-01 22:22:30.731323096,21.003161
5,3,2017-01-01 22:22:28.731323096,2017-01-01 22:22:44.587000000,15.855676


In [490]:
frame310_1 = pd.Interval(frames[frames['n'] == 310]['start'].iloc[0], frames[frames['n'] == 310]['stop'].iloc[0], closed='both')
frame310_1

Interval('2017-01-01 22:21:31.756918332', '2017-01-01 22:21:52.725000', closed='both')

## TOM_TDS - OLD

### Load data from SaVoir simulation

In [3]:
simulation = pd.read_excel('Export_TOM_Full_GC_S1_S2_S3_RO_S1SRM.xlsx')

# add DTID
simulation['DTID'] = [int(tm.DataTakeID(row.Orbit, row.TAnx).tobytes().hex(), 16) for row in simulation.itertuples()]
# add mcid, rcid, track
orbits = [AbsoluteOrbit(n, tm.MissionPhase.TOM) for n in simulation['Orbit']]
simulation['mcid'] = [o.mcid() for o in orbits]
simulation['rcid'] = [o.rcid() for o in orbits]
simulation['track'] = [o.track() for o in orbits]

simulation.sort_values(by=['Start'], inplace=True)

simulation.head()

,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,NW_Lat,...,SW_Lat,SW_Lon,Orbit,Anx,AnxLon,TAnx,DTID,mcid,rcid,track
0,2017-01-01 06:01:31.394,2017-01-01 06:03:44.504,133.110305,BIOMASS,Stripmap S1,53.961516,915.099690,49580.620718,ASCENDING,13.001162,...,4.907343,-3.881037,1,2017-01-01 06:00:01.272,1.849826e-07,90.122152,8282,1,1,1
1,2017-01-01 06:08:39.801,2017-01-01 06:36:30.245,1670.444243,BIOMASS,Receive Only,55.066325,11519.783482,630257.214895,ASCENDING,44.859358,...,30.879652,-10.287763,1,2017-01-01 06:00:01.272,-2.086955e-08,518.528527,8710,1,1,1
2,2017-01-01 06:52:49.307,2017-01-01 06:55:03.719,134.411778,BIOMASS,Stripmap S1,54.464226,919.977565,50248.148728,DESCENDING,-14.293988,...,-22.460572,165.421474,1,2017-01-01 06:00:01.272,6.098976e-09,3168.035430,11360,1,1,1
3,2017-01-01 07:49:10.607,2017-01-01 08:11:51.127,1360.520507,BIOMASS,Receive Only,54.972860,9388.170636,514126.843406,ASCENDING,54.860562,...,39.278476,-37.584445,2,2017-01-01 07:38:12.217,-2.454561e+01,658.389814,17042,1,1,2
4,2017-01-01 08:28:10.480,2017-01-01 08:29:15.351,64.870512,BIOMASS,Stripmap S1,53.780686,444.955991,24185.787974,DESCENDING,-3.962771,...,-7.911707,143.907193,2,2017-01-01 07:38:12.217,-2.454561e+01,2998.263258,19382,1,1,2


### MM Scenario

#### Data Take

In [4]:
# Find acquisitions across ANX
[n for n in range(simulation.shape[0] - 1)
 if simulation.iloc[n]['End'] > simulation.iloc[n + 1]['Anx'] and
 simulation.iloc[n]['Anx'] < simulation.iloc[n + 1]['Anx'] and
 simulation.iloc[n]['SensorMode'] == simulation.iloc[n + 1]['SensorMode'] and
 simulation.iloc[n]['SensorMode'] != 'Receive Only'][:4]

[51, 112, 119, 132]

In [5]:
simulation[['Anx', 'Start', 'End', 'Duration', 'SensorMode', 'Orbit', 'mcid', 'rcid', 'track']].iloc[51:53]

,Anx,Start,End,Duration,SensorMode,Orbit,mcid,rcid,track
51,2017-01-01 20:43:39.780,2017-01-01 22:21:04.727,2017-01-01 22:22:44.587,99.859843,Stripmap S1,10,1,1,10
52,2017-01-01 22:21:50.725,2017-01-01 22:25:11.726,2017-01-01 22:26:26.213,74.486619,Stripmap S1,11,1,1,11


ID of selected acquisition: 51

In [6]:
data_take = pd.Interval(simulation.loc[51, 'Start'], simulation.loc[51, 'End'], closed='both')
data_take

Interval('2017-01-01 22:21:04.727000', '2017-01-01 22:22:44.587000', closed='both')

In [7]:
dtid = simulation.loc[51, 'DTID']
dtid

87764

#### Swath

In [8]:
swath_id = simulation.loc[51]['SensorMode'][-2:]
swath_id

'S1'

#### Mission phase, global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [9]:
mission_phase = 'TOMOGRAPHIC'
gcid = 1
mcid = simulation.loc[51]['mcid']
rcid = simulation.loc[51]['rcid']
drift = False
orbit_numbers = simulation.loc[51:52]['Orbit'].to_numpy()
track_numbers = simulation.loc[51:52]['track'].to_numpy()

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [10]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard

#### Affected Orbits

Orbits overlapping data take.

In [11]:
orbits = [i for i in
          (pd.Interval(simulation.iloc[n]['Anx'], simulation.iloc[n + 1]['Anx'], closed='both')
           for n in range(simulation.shape[0] - 1)
           if simulation.iloc[n]['Anx'] < simulation.iloc[n + 1]['Anx'] and
           simulation.iloc[n]['Orbit'] < simulation.iloc[n + 1]['Orbit'])
          if i.overlaps(data_take)]
orbits

[Interval('2017-01-01 20:43:39.780000', '2017-01-01 22:21:50.725000', closed='both'),
 Interval('2017-01-01 22:21:50.725000', '2017-01-02 00:00:01.670000', closed='both')]

#### Footprint

In [12]:
dt_polygon = simulation.loc[6, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Level 0 slices

Slice intervals overlapping data take:

In [13]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,62,2017-01-01 22:21:04.727,2017-01-01 22:21:57.725,52.998
1,1,2017-01-01 22:21:45.725,2017-01-01 22:22:44.587,58.862


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [14]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,62,2017-01-01 22:20:48.727,2017-01-01 22:21:57.725,68.998
1,1,2017-01-01 22:21:45.725,2017-01-01 22:22:44.587,58.862


#### Level 1 frames

Frame intervals overlapping data take:

In [15]:
frames = []
for orbit in orbits:
    fg = Grid(orbit.left, orbit.right, const.FRAME_GRID_DURATION, const.FRAME_INITIAL_OVERLAP, const.FRAME_FINAL_OVERLAP, const.NUM_FRAMES)
    frames.extend(fg.intervals(data_take, 7))

frames = pd.DataFrame(
    [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
     for f in frames]
)
frames

,n,start,stop,duration
0,308,2017-01-01 22:21:04.727000000,2017-01-01 22:21:14.753756784,10.026756
1,309,2017-01-01 22:21:12.753756784,2017-01-01 22:21:33.756918332,21.003161
2,310,2017-01-01 22:21:31.756918332,2017-01-01 22:21:52.725000000,20.968081
3,1,2017-01-01 22:21:50.725000000,2017-01-01 22:22:11.728161548,21.003161
4,2,2017-01-01 22:22:09.728161548,2017-01-01 22:22:30.731323096,21.003161
5,3,2017-01-01 22:22:28.731323096,2017-01-01 22:22:44.587000000,15.855676


#### Raw Data generation

Scenario generation

In [16]:
baseline = 1

config = {
    'mission': 'biomass',
    'scenarios': []
}

# Add MM scenario
config['scenarios'].append(
    {
        'name': 'TOM_TDS - MM',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': acquisition_station,
        'baseline': baseline,
        'outputs': [
            {
                'file_type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [25, 26]
        ]
    }
)

In [17]:
with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [20]:
baseline = 1

# L0 Step 1
config = {
    'mission': 'biomass',
    'scenarios': [
        {
            'name': f'TOM_TDS - L0PFS1-MM-{file_type}',
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{file_type[5:7]}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{file_type[5:7]}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'file_type': file_type,
                    'metadata_source': f'.*{file_type}_.*'
                }
            ]
        }
        for file_type in ('RAW_022_10', 'RAW_023_10', 'RAW_024_10', 'RAW_025_10', 'RAW_026_10')
    ]
}

# L0 Step 2
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS2-MM',
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0S',
                'metadata_source': '.*RAW_025_10_.*'
            },
            {
                'file_type': f'{swath_id}_RAW__0M',
                'metadata_source': '.*RAW_025_10_.*'
            }
        ]
    }
)

# L0 Step 3
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS3-MM',
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z'
            }
        ]
    }
)

# L0 Step 4
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS4',
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(rcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

# L0 Step 6
config['scenarios'].append(
    {
        'name': 'TOM_TDS - OAPF',
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': 'SM'
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)


In [21]:
with open('tom-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1 a/b

Scenario generation

In [22]:
baseline = 1

# Level 1 a/b
config = {
    'mission': 'biomass',
    'scenarios': [
        {
            'name': f'TOM_TDS - L1PF-MM-slice-{slice_number}',
            'file_name': 'l1pf.sh',
            'processor_name': 'L1PF',
            'processor_version': '01.00',
            'task_name': 'L1PF',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'data_takes': [
                {
                    'data_take_id': str(dtid),
                    'start': data_take.left.isoformat() + 'Z',
                    'stop': data_take.right.isoformat() + 'Z',
                    'swath': swath_id,
                    'operational_mode': 'SM'
                }
            ],
            'mission_phase': mission_phase,
            'global_coverage_id': str(gcid),
            'major_cycle_id': str(gcid),
            'repeat_cycle_id': str(gcid),
            'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
            'footprint_polygon': footprint_polygon,
            'outputs': [
                {
                    'file_type': 'S3_SCS__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': 'S3_SCS__1M',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                }
            ]
        }
        for slice_number in slices['n']
    ]
}


In [23]:
with open('tom-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### RO Scenario

#### Data Take

In [24]:
simulation[['Anx', 'Start', 'End', 'Duration', 'SensorMode']].head()

,Anx,Start,End,Duration,SensorMode
0,2017-01-01 06:00:01.272,2017-01-01 06:01:31.394,2017-01-01 06:03:44.504,133.110305,Stripmap S1
1,2017-01-01 06:00:01.272,2017-01-01 06:08:39.801,2017-01-01 06:36:30.245,1670.444243,Receive Only
2,2017-01-01 06:00:01.272,2017-01-01 06:52:49.307,2017-01-01 06:55:03.719,134.411778,Stripmap S1
3,2017-01-01 07:38:12.217,2017-01-01 07:49:10.607,2017-01-01 08:11:51.127,1360.520507,Receive Only
4,2017-01-01 07:38:12.217,2017-01-01 08:28:10.480,2017-01-01 08:29:15.351,64.870512,Stripmap S1


ID of selected acquisition: 1

In [25]:
data_take = pd.Interval(simulation.loc[1, 'Start'], simulation.loc[1, 'End'], closed='both')
data_take

Interval('2017-01-01 06:08:39.801000', '2017-01-01 06:36:30.245000', closed='both')

In [26]:
dtid = simulation.loc[1, 'DTID']
dtid

8710

#### Swath

In [27]:
swath_id = 'RO'

#### Other

In [39]:
mission_phase = 'TOMOGRAPHIC'
gcid = 1
mcid = simulation.loc[1]['mcid']
rcid = simulation.loc[1]['rcid']
drift = False
track_number = simulation.loc[1]['track']

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [29]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard

#### Affected Orbits

Orbits overlapping data take.

In [30]:
orbits = [i for i in
          (pd.Interval(simulation.iloc[n]['Anx'], simulation.iloc[n + 1]['Anx'], closed='both')
           for n in range(simulation.shape[0] - 1)
           if simulation.iloc[n]['Anx'] < simulation.iloc[n + 1]['Anx'] and
           simulation.iloc[n]['Orbit'] < simulation.iloc[n + 1]['Orbit'])
          if i.overlaps(data_take)]
orbits

[Interval('2017-01-01 06:00:01.272000', '2017-01-01 07:38:12.217000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [31]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,6,2017-01-01 06:08:39.801000000,2017-01-01 06:09:38.366846440,58.565846
1,7,2017-01-01 06:09:26.366846440,2017-01-01 06:11:13.382654180,107.015807
2,8,2017-01-01 06:11:01.382654180,2017-01-01 06:12:48.398461920,107.015807
3,9,2017-01-01 06:12:36.398461920,2017-01-01 06:14:23.414269660,107.015807
4,10,2017-01-01 06:14:11.414269660,2017-01-01 06:15:58.430077400,107.015807
5,11,2017-01-01 06:15:46.430077400,2017-01-01 06:17:33.445885140,107.015807
6,12,2017-01-01 06:17:21.445885140,2017-01-01 06:19:08.461692880,107.015807
7,13,2017-01-01 06:18:56.461692880,2017-01-01 06:20:43.477500620,107.015807
8,14,2017-01-01 06:20:31.477500620,2017-01-01 06:22:18.493308360,107.015807
9,15,2017-01-01 06:22:06.493308360,2017-01-01 06:23:53.509116100,107.015807


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [32]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,6,2017-01-01 06:08:23.801000000,2017-01-01 06:09:38.366846440,74.565846
1,7,2017-01-01 06:09:26.366846440,2017-01-01 06:11:13.382654180,107.015807
2,8,2017-01-01 06:11:01.382654180,2017-01-01 06:12:48.398461920,107.015807
3,9,2017-01-01 06:12:36.398461920,2017-01-01 06:14:23.414269660,107.015807
4,10,2017-01-01 06:14:11.414269660,2017-01-01 06:15:58.430077400,107.015807
5,11,2017-01-01 06:15:46.430077400,2017-01-01 06:17:33.445885140,107.015807
6,12,2017-01-01 06:17:21.445885140,2017-01-01 06:19:08.461692880,107.015807
7,13,2017-01-01 06:18:56.461692880,2017-01-01 06:20:43.477500620,107.015807
8,14,2017-01-01 06:20:31.477500620,2017-01-01 06:22:18.493308360,107.015807
9,15,2017-01-01 06:22:06.493308360,2017-01-01 06:23:53.509116100,107.015807


#### Raw Data generation

Scenario generation

In [33]:
with open('tom-tds-raw-gen-scenarios.json') as fh:
    config = json.load(fh)

In [40]:
baseline = 1

# Add RO scenario
config['scenarios'].append(
    {
        'name': 'TOM_TDS - RO',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': acquisition_station,
        'baseline': baseline,
        'outputs': [
            {
                'file_type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [27, 28]
        ]
    }
)

In [41]:
with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [36]:
with open('tom-tds-l0-scenarios.json') as fh:
    config = json.load(fh)

In [42]:
baseline = 1

# L0 Step 1
config = {
    'mission': 'biomass',
    'scenarios': [
        {
            'name': f'TOM_TDS - L0PFS1-RO-{file_type}',
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{file_type[5:7]}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{file_type[5:7]}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'file_type': file_type,
                    'metadata_source': f'.*{file_type}_.*'
                }
            ]
        }
        for file_type in ('RAW_022_10', 'RAW_023_10', 'RAW_024_10', 'RAW_027_10', 'RAW_028_10')
    ]
}

# L0 Step 2
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS2-RO',
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_number),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'RO_RAW__0S',
                'metadata_source': '.*RAW_027_10_.*'
            }
        ]
    }
)

# L0 Step 3
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS3-RO',
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_number),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'S3_RAWP_0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z'
            }
        ]
    }
)

# L0 Step 6
config['scenarios'].append(
    {
        'name': 'TOM_TDS - OAPF-RO',
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': 'SM'
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)


In [43]:
with open('tom-tdsl0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### EC Scenario

#### Data Take

In [58]:
simulation.query("SensorMode == 'Stripmap S1' and 60 <= Duration < 61").index

Int64Index([ 3751,  7424,  7620,  7816, 10998, 11196, 11395, 11594, 11793,
            11992, 12191, 18342, 18539, 22003, 22202, 22401, 22601, 22803,
            23005, 23207],
           dtype='int64')

ID of selected acquisition: 3751

In [61]:
simulation.loc[3751:3755][['Anx', 'Start', 'End', 'Duration', 'SensorMode']]

,Anx,Start,End,Duration,SensorMode
3751,2017-03-15 06:38:08.068,2017-03-15 08:14:03.108,2017-03-15 08:15:04.102,60.994296,Stripmap S1
3752,2017-03-15 08:16:19.013,2017-03-15 08:24:28.793,2017-03-15 08:48:37.624,1448.831072,Receive Only
3753,2017-03-15 08:16:19.013,2017-03-15 09:05:54.619,2017-03-15 09:06:26.619,32.000000,Stripmap S1
3754,2017-03-15 08:16:19.013,2017-03-15 09:08:25.964,2017-03-15 09:11:13.886,167.922205,Stripmap S1
3755,2017-03-15 08:16:19.013,2017-03-15 09:45:10.067,2017-03-15 09:57:34.808,744.741470,Stripmap S1


In [62]:
data_take = pd.Interval(simulation.loc[3751, 'Start'], simulation.loc[3751, 'End'], closed='both')
data_take

Interval('2017-03-15 08:14:03.108000', '2017-03-15 08:15:04.102000', closed='both')

In [63]:
dtid = simulation.loc[3751, 'DTID']
dtid

8787579

#### Swath

In [64]:
swath_id = 'EC'

#### Other

In [65]:
mission_phase = 'TOMOGRAPHIC'
gcid = 1
mcid = simulation.loc[3751]['mcid']
rcid = simulation.loc[3751]['rcid']
drift = False
orbit_numbers = simulation.loc[3751]['Orbit']
track_numbers = simulation.loc[3751]['track']

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [66]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard

#### Affected Orbits

Orbits overlapping data take.

In [67]:
orbits = [i for i in
          (pd.Interval(simulation.iloc[n]['Anx'], simulation.iloc[n + 1]['Anx'], closed='both')
           for n in range(simulation.shape[0] - 1)
           if simulation.iloc[n]['Anx'] < simulation.iloc[n + 1]['Anx'] and
           simulation.iloc[n]['Orbit'] < simulation.iloc[n + 1]['Orbit'])
          if i.overlaps(data_take)]
orbits

[Interval('2017-03-15 06:38:08.068000', '2017-03-15 08:16:19.013000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [68]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,61,2017-03-15 08:14:03.108,2017-03-15 08:15:04.102,60.994


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [69]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,61,2017-03-15 08:13:47.108,2017-03-15 08:15:04.102,76.994


#### Level 1 frames

Frame intervals overlapping data take:

In [70]:
frames = []
for orbit in orbits:
    fg = Grid(orbit.left, orbit.right, const.FRAME_GRID_DURATION, const.FRAME_INITIAL_OVERLAP, const.FRAME_FINAL_OVERLAP, const.NUM_FRAMES)
    frames.extend(fg.intervals(data_take, 7))

frames = pd.DataFrame(
    [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
     for f in frames]
)
frames

,n,start,stop,duration
0,304,2017-03-15 08:14:03.108000000,2017-03-15 08:14:27.029110592,23.921110
1,305,2017-03-15 08:14:25.029110592,2017-03-15 08:14:46.032272140,21.003161
2,306,2017-03-15 08:14:44.032272140,2017-03-15 08:15:04.102000000,20.069727


#### Raw Data generation

Scenario generation

In [33]:
with open('tom-tds-raw-gen-scenarios.json') as fh:
    config = json.load(fh)

In [73]:
baseline = 1

# Add EC scenario
config['scenarios'].append(
    {
        'name': 'TOM_TDS - EC',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': acquisition_station,
        'baseline': baseline,
        'outputs': [
            {
                'file_type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [35, 36]
        ]
    }
)

In [74]:
with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [20]:
baseline = 1

# L0 Step 1
config = {
    'mission': 'biomass',
    'scenarios': [
        {
            'name': f'TOM_TDS - L0PFS1-EC-{file_type}',
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{file_type[5:7]}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{file_type[5:7]}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'file_type': file_type,
                    'metadata_source': f'.*{file_type}_.*'
                }
            ]
        }
        for file_type in ('RAW_022_10', 'RAW_023_10', 'RAW_024_10', 'RAW_035_10', 'RAW_036_10')
    ]
}

# L0 Step 2
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS2-EC',
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'EC'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'EC_RAWP_0S',
                'metadata_source': '.*RAW_035_10_.*'
            },
            {
                'file_type': 'EC_RAWP_0M',
                'metadata_source': '.*RAW_035_10_.*'
            }
        ]
    }
)

# L0 Step 3
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS3-EC',
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'EC'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'EC_RAWP_0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z'
            }
        ]
    }
)

# L0 Step 4
config['scenarios'].append(
    {
        'name': 'TOM_TDS - L0PFS4',
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(rcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track_numbers[0]),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

# L0 Step 6
config['scenarios'].append(
    {
        'name': 'TOM_TDS - OAPF',
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': 'SM'
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)


In [21]:
with open('tom-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

## INT_TDS

### Load data from SaVoir simulation

In [401]:
def next_anx(row):
    next_orbit = row['Orbit'] + 1
    next_anx = row['Anx'] + pd.to_timedelta(ORBIT_DURATION, 's')
    r = simulation.query("Orbit == @next_orbit")
    if r.shape[0] > 0:
        next_anx = r['Anx'].min()
    return next_anx

simulation = pd.read_excel('Export_INT_Full_GC_S1_S2_S3_RO_S1SRM.xlsx')

# add DTID
simulation['DTID'] = [int(tm.DataTakeID(row.Orbit, row.TAnx).tobytes().hex(), 16) for row in simulation.itertuples()]
# add mcid, rcid, track, ext anx
orbits = [AbsoluteOrbit(n, tm.MissionPhase.INT) for n in simulation['Orbit']]
simulation['gcid'] = [o.gcid() for o in orbits]
simulation['mcid'] = [o.mcid() for o in orbits]
simulation['rcid'] = [o.rcid() for o in orbits]
simulation['track'] = [o.track() for o in orbits]
simulation['NextAnx'] = simulation.apply(next_anx, axis=1)

simulation.sort_values(by=['Start'], inplace=True)

simulation.columns

Index(['Start', 'End', 'Duration', 'Satellite', 'SensorMode', 'Width',
       'Length', 'SwathArea', 'Pass', 'NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon',
       'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'Orbit', 'Anx', 'AnxLon',
       'TAnx', 'DTID', 'gcid', 'mcid', 'rcid', 'track', 'NextAnx'],
      dtype='object')

### MM scenario

#### Data Take selection

Select Data Take across ANX

In [402]:
simulation.query("End > NextAnx and SensorMode != 'Receive Only'")[['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'SensorMode']].head(10)

,Anx,Start,End,NextAnx,Duration,SensorMode
8,2017-01-01 07:38:12.253,2017-01-01 09:09:51.681,2017-01-01 09:18:02.746,2017-01-01 09:16:23.235,491.064347,Stripmap S1
18,2017-01-01 09:16:23.235,2017-01-01 10:50:05.283,2017-01-01 10:56:37.863,2017-01-01 10:54:34.216,392.579435,Stripmap S1
52,2017-01-01 20:43:40.105,2017-01-01 22:21:05.081,2017-01-01 22:22:44.939,2017-01-01 22:21:51.087,99.857472,Stripmap S1
65,2017-01-02 01:38:13.050,2017-01-02 03:15:11.863,2017-01-02 03:27:17.657,2017-01-02 03:16:24.031,725.793887,Stripmap S1
68,2017-01-02 03:16:24.031,2017-01-02 04:45:18.018,2017-01-02 04:58:18.230,2017-01-02 04:54:35.013,780.211966,Stripmap S1
82,2017-01-02 08:10:56.976,2017-01-02 09:40:05.687,2017-01-02 09:52:17.876,2017-01-02 09:49:07.957,732.189394,Stripmap S1
117,2017-01-02 19:38:13.846,2017-01-02 21:15:26.566,2017-01-02 21:17:08.927,2017-01-02 21:16:24.828,102.361579,Stripmap S1
124,2017-01-02 21:16:24.828,2017-01-02 22:52:35.625,2017-01-02 22:56:27.166,2017-01-02 22:54:35.810,231.541100,Stripmap S1
135,2017-01-03 02:10:57.773,2017-01-03 03:42:34.941,2017-01-03 03:58:22.498,2017-01-03 03:49:08.755,947.556683,Stripmap S1
138,2017-01-03 03:49:08.755,2017-01-03 05:26:49.620,2017-01-03 05:27:21.620,2017-01-03 05:27:19.736,32.000000,Stripmap S1


ID of selected acquisition: 124

In [403]:
acq_idx = 124

In [404]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-02 22:52:35.625000', '2017-01-02 22:56:27.166000', closed='both')

In [405]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

210570

#### Swath

In [408]:
# get swath id from SensorMode name
swath_id = simulation.loc[acq_idx]['SensorMode'][-2:]
swath_id

'S1'

In [409]:
# crosscheck with orbit number
AbsoluteOrbit(simulation.loc[acq_idx]['Orbit'], tm.MissionPhase.INT).swathid()

1

#### Mission phase, global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [410]:
mission_phase = 'INTERFEROMETRIC'
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [411]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [412]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-02 23:26:27.166000')

#### Affected Orbits

Orbits overlapping data take.

In [413]:
orbits = [
    pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx'], closed='both'),
    pd.Interval(
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['Anx'].min(),
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['NextAnx'].min(),
        closed='both'
    )
]
orbits

[Interval('2017-01-02 21:16:24.828000', '2017-01-02 22:54:35.810000', closed='both'),
 Interval('2017-01-02 22:54:35.810000', '2017-01-03 00:32:46.792000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [414]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,61,2017-01-02 22:52:35.625000000,2017-01-02 22:53:07.792272140,32.167272
1,62,2017-01-02 22:52:55.792272140,2017-01-02 22:54:42.810000000,107.017727
2,1,2017-01-02 22:54:30.810000000,2017-01-02 22:56:27.166000000,116.356000


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [415]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,61,2017-01-02 22:52:19.625000000,2017-01-02 22:53:07.792272140,48.167272
1,62,2017-01-02 22:52:55.792272140,2017-01-02 22:54:42.810000000,107.017727
2,1,2017-01-02 22:54:30.810000000,2017-01-02 22:56:27.166000000,116.356000


#### Level 1 frames

Frame intervals overlapping data take:

In [416]:
frames = []
for orbit in orbits:
    fg = Grid(orbit.left, orbit.right, const.FRAME_GRID_DURATION, const.FRAME_INITIAL_OVERLAP, const.FRAME_FINAL_OVERLAP, const.NUM_FRAMES)
    frames.extend(fg.intervals(data_take, 7))

frames = pd.DataFrame(
    [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
     for f in frames]
)
frames

,n,start,stop,duration
0,304,2017-01-02 22:52:35.625000000,2017-01-02 22:52:43.789110592,8.164110
1,305,2017-01-02 22:52:41.789110592,2017-01-02 22:53:02.792272140,21.003161
2,306,2017-01-02 22:53:00.792272140,2017-01-02 22:53:21.795433688,21.003161
3,307,2017-01-02 22:53:19.795433688,2017-01-02 22:53:40.798595236,21.003161
4,308,2017-01-02 22:53:38.798595236,2017-01-02 22:53:59.801756784,21.003161
5,309,2017-01-02 22:53:57.801756784,2017-01-02 22:54:18.804918332,21.003161
6,310,2017-01-02 22:54:16.804918332,2017-01-02 22:54:37.810000000,21.005081
7,1,2017-01-02 22:54:35.810000000,2017-01-02 22:54:56.813161548,21.003161
8,2,2017-01-02 22:54:54.813161548,2017-01-02 22:55:15.816323096,21.003161
9,3,2017-01-02 22:55:13.816323096,2017-01-02 22:55:34.819484644,21.003161


In [417]:
frame310_1 = pd.Interval(frames[frames['n'] == 310]['start'].iloc[0], frames[frames['n'] == 310]['stop'].iloc[0], closed='both')
frame310_1

Interval('2017-01-02 22:54:16.804918332', '2017-01-02 22:54:37.810000', closed='both')

#### Additional data takes for stack

Selected slice and frame:
- slice: 62
- frame: 310

Select additional data takes having the same global coverage id, major cycle id, swath (sensor mode) and track:

In [406]:
sensor_mode = simulation.loc[acq_idx]['SensorMode']
track = simulation.loc[acq_idx]['track']
result = simulation.query("gcid == @gcid and mcid == @mcid and SensorMode == @sensor_mode and track == @track")
# discard orbit of previously selected data take
orbit = simulation.loc[acq_idx]['Orbit']
result = result.query("Orbit != @orbit")
result

,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,NW_Lat,...,Orbit,Anx,AnxLon,TAnx,DTID,gcid,mcid,rcid,track,NextAnx
333,2017-01-05 21:18:34.988,2017-01-05 21:20:05.589,90.601362,BIOMASS,Stripmap S1,54.028089,622.940026,33747.537424,ASCENDING,12.657015,...,69,2017-01-05 21:16:28.015,130.888561,126.972588,565374,1,1,2,25,2017-01-05 22:54:38.996
334,2017-01-05 21:24:21.464,2017-01-05 21:37:41.280,799.816052,BIOMASS,Stripmap S1,54.554738,5518.957087,300984.406244,ASCENDING,73.819388,...,69,2017-01-05 21:16:28.015,130.888561,473.448612,565721,1,1,2,25,2017-01-05 22:54:38.996
336,2017-01-05 22:03:35.647,2017-01-05 22:20:33.929,1018.281106,BIOMASS,Stripmap S1,54.667486,6957.553586,382014.451679,DESCENDING,6.425647,...,69,2017-01-05 21:16:28.015,130.888561,2827.632383,568075,1,1,2,25,2017-01-05 22:54:38.996
337,2017-01-05 22:21:52.575,2017-01-05 22:22:36.225,43.649862,BIOMASS,Stripmap S1,55.978226,296.654255,16587.449433,DESCENDING,-59.883987,...,69,2017-01-05 21:16:28.015,130.888561,3924.559716,569172,1,1,2,25,2017-01-05 22:54:38.996
338,2017-01-05 22:23:51.052,2017-01-05 22:36:50.170,779.118494,BIOMASS,Stripmap S1,56.383350,5282.550952,296047.877947,DESCENDING,-65.048952,...,69,2017-01-05 21:16:28.015,130.888561,4043.036644,569291,1,1,2,25,2017-01-05 22:54:38.996
339,2017-01-05 22:53:10.505,2017-01-05 22:56:30.450,199.944862,BIOMASS,Stripmap S1,54.081772,1372.848808,74450.497930,ASCENDING,6.205076,...,69,2017-01-05 21:16:28.015,130.888561,5802.489952,571050,1,1,2,25,2017-01-05 22:54:38.996
547,2017-01-08 21:18:37.804,2017-01-08 21:20:08.966,91.162288,BIOMASS,Stripmap S1,54.028090,626.796228,33952.593846,ASCENDING,12.668616,...,113,2017-01-08 21:16:31.200,130.875297,126.603601,925822,1,1,3,25,2017-01-08 22:54:42.181
548,2017-01-08 21:24:24.604,2017-01-08 21:37:44.472,799.868344,BIOMASS,Stripmap S1,54.541065,5519.317595,300970.854370,ASCENDING,73.819669,...,113,2017-01-08 21:16:31.200,130.875297,473.403580,926169,1,1,3,25,2017-01-08 22:54:42.181
550,2017-01-08 22:03:38.668,2017-01-08 22:20:37.046,1018.378327,BIOMASS,Stripmap S1,54.678689,6958.226281,382063.769643,DESCENDING,6.435728,...,113,2017-01-08 21:16:31.200,130.875297,2827.467666,928523,1,1,3,25,2017-01-08 22:54:42.181
551,2017-01-08 22:21:55.759,2017-01-08 22:22:38.996,43.237173,BIOMASS,Stripmap S1,55.978233,293.849957,16430.560879,DESCENDING,-59.883879,...,113,2017-01-08 21:16:31.200,130.875297,3924.558901,929620,1,1,3,25,2017-01-08 22:54:42.181


Select data takes having overlap with frame 310:

In [407]:
def overlap(data_take_start, data_take_stop, orbit_start, orbit_stop, frame_number):
    data_take = pd.Interval(data_take_start, data_take_stop, closed='both')
    frame_grid = Grid(
        orbit_start,
        orbit_stop,
        const.FRAME_GRID_DURATION,
        const.FRAME_INITIAL_OVERLAP,
        const.FRAME_FINAL_OVERLAP,
        const.NUM_FRAMES
    )
    return frame_grid.interval(frame_number).overlaps(data_take)

idx = result.apply(
    lambda r: overlap(r['Start'], r['End'], r['Anx'], r['NextAnx'], 310),
    axis=1
)
result = result[idx]
result[['Orbit', 'Anx', 'Start', 'End', 'NextAnx', 'Duration']]

,Orbit,Anx,Start,End,NextAnx,Duration
339,69,2017-01-05 21:16:28.015,2017-01-05 22:53:10.505,2017-01-05 22:56:30.450,2017-01-05 22:54:38.996,199.944862
553,113,2017-01-08 21:16:31.200,2017-01-08 22:53:13.688,2017-01-08 22:56:33.734,2017-01-08 22:54:42.181,200.045306


#### Raw Data generation

Scenario generation

In [418]:
config = {
    'mission': 'biomass',
    'scenarios': []
}

In [419]:
config['scenarios'].append(
    {
        'name': 'INT_TDS - DT1',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': acquisition_station,
        'baseline': BASELINE,
        'outputs': [
            {
                'file_type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [25, 26]
        ]
    }
)

with open('int-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [420]:
config = {
    'mission': 'biomass',
    'scenarios': []
}

In [421]:
# L0 Step 1
config['scenarios'].extend(
    [
        {
            'name': f'INT_TDS - DT1 - L0PFS1 - {file_type}',
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{file_type[5:7]}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{file_type[5:7]}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': BASELINE,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'file_type': file_type,
                    'metadata_source': f'.*{file_type}_.*'
                }
            ]
        }
        for file_type in ('RAW_022_10', 'RAW_023_10', 'RAW_024_10', 'RAW_025_10', 'RAW_026_10')
    ]
)

# L0 Step 2
config['scenarios'].append(
    {
        'name': 'INT_TDS - DT1 - L0PFS2',
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0S',
                'metadata_source': '.*RAW_025_10_.*'
            },
            {
                'file_type': f'{swath_id}_RAWP_0M',
                'metadata_source': '.*RAW_025_10_.*'
            }
        ]
    }
)

# L0 Step 3
config['scenarios'].append(
    {
        'name': 'INT_TDS - DT1 - L0PFS3',
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',                
            }
        ]
    }
)

# L0 Step 4
config['scenarios'].append(
    {
        'name': 'INT_TDS - DT1 - L0PFS4',
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(rcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

# L0 Step 6
config['scenarios'].append(
    {
        'name': 'INT_TDS - DT1 - OAPF',
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': 'SM'
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

with open('int-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1a/b

In [422]:
config = {
    'mission': 'biomass',
    'scenarios': []
}

In [423]:
# Level 1 a/b
config['scenarios'].extend(
    [
        {
            'name': f'INT_TDS - DT1 - L1PF - slice {slice_number}',
            'file_name': 'l1pf.sh',
            'processor_name': 'L1PF',
            'processor_version': '01.00',
            'task_name': 'L1PF',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': BASELINE,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'data_takes': [
                {
                    'data_take_id': str(dtid),
                    'start': data_take.left.isoformat() + 'Z',
                    'stop': data_take.right.isoformat() + 'Z',
                    'swath': swath_id,
                    'operational_mode': 'SM'
                }
            ],
            'mission_phase': mission_phase,
            'global_coverage_id': str(gcid),
            'major_cycle_id': str(gcid),
            'repeat_cycle_id': str(gcid),
            'track_nr': str(track),  # ISSUE: track_nr should be a list
            'footprint_polygon': footprint_polygon,
            'outputs': [
                {
                    'file_type': f'{swath_id}_SCS__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_SCS__1M',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_DGM__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                }
            ]
        }
        for slice_number in slices['n']
    ]
)

In [424]:
with open('int-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### MM - Stack - second data take

In [425]:
acq_idx = 339

In [426]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-05 22:53:10.505000', '2017-01-05 22:56:30.450000', closed='both')

In [427]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

571050

#### Mission phase, global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [428]:
mission_phase = 'INTERFEROMETRIC'
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [429]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [430]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-05 23:26:30.450000')

#### Affected Orbits

Orbits overlapping data take.

In [431]:
orbits = [
    pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx'], closed='both'),
    pd.Interval(
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['Anx'].min(),
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['NextAnx'].min(),
        closed='both'
    )
]
orbits

[Interval('2017-01-05 21:16:28.015000', '2017-01-05 22:54:38.996000', closed='both'),
 Interval('2017-01-05 22:54:38.996000', '2017-01-06 00:32:49.978000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [432]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,62,2017-01-05 22:53:10.505,2017-01-05 22:54:45.996,95.491
1,1,2017-01-05 22:54:33.996,2017-01-05 22:56:30.450,116.454


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [433]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,62,2017-01-05 22:52:54.505,2017-01-05 22:54:45.996,111.491
1,1,2017-01-05 22:54:33.996,2017-01-05 22:56:30.450,116.454


#### Level 1 frames

Frame intervals overlapping data take:

In [434]:
frames = []
for orbit in orbits:
    fg = Grid(orbit.left, orbit.right, const.FRAME_GRID_DURATION, const.FRAME_INITIAL_OVERLAP, const.FRAME_FINAL_OVERLAP, const.NUM_FRAMES)
    frames.extend(fg.intervals(data_take, 7))

frames = pd.DataFrame(
    [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
     for f in frames]
)
frames

,n,start,stop,duration
0,306,2017-01-05 22:53:10.505000000,2017-01-05 22:53:24.982433688,14.477433
1,307,2017-01-05 22:53:22.982433688,2017-01-05 22:53:43.985595236,21.003161
2,308,2017-01-05 22:53:41.985595236,2017-01-05 22:54:02.988756784,21.003161
3,309,2017-01-05 22:54:00.988756784,2017-01-05 22:54:21.991918332,21.003161
4,310,2017-01-05 22:54:19.991918332,2017-01-05 22:54:40.996000000,21.004081
5,1,2017-01-05 22:54:38.996000000,2017-01-05 22:54:59.999161548,21.003161
6,2,2017-01-05 22:54:57.999161548,2017-01-05 22:55:19.002323096,21.003161
7,3,2017-01-05 22:55:17.002323096,2017-01-05 22:55:38.005484644,21.003161
8,4,2017-01-05 22:55:36.005484644,2017-01-05 22:55:57.008646192,21.003161
9,5,2017-01-05 22:55:55.008646192,2017-01-05 22:56:16.011807740,21.003161


In [435]:
frame310_2 = pd.Interval(frames[frames['n'] == 310]['start'].iloc[0], frames[frames['n'] == 310]['stop'].iloc[0], closed='both')
frame310_2

Interval('2017-01-05 22:54:19.991918332', '2017-01-05 22:54:40.996000', closed='both')

#### Raw Data generation

Scenario generation

In [436]:
with open('int-tds-raw-gen-scenarios.json') as fh:
    config = json.load(fh)

In [437]:
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT2',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': acquisition_station,
        'baseline': BASELINE,
        'outputs': [
            {
                'file_type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [25, 26]
        ]
    }
)

In [438]:
with open('int-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [439]:
with open('int-tds-l0-scenarios.json') as fh:
    config = json.load(fh)

In [440]:
# L0 Step 1
config['scenarios'].extend(
    [
        {
            'name': f'INT_TDS - DT2 - L0PFS1 - {file_type}',
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{file_type[5:7]}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{file_type[5:7]}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': BASELINE,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'file_type': file_type,
                    'metadata_source': f'.*{file_type}_.*'
                }
            ]
        }
        for file_type in ('RAW_022_10', 'RAW_023_10', 'RAW_024_10', 'RAW_025_10', 'RAW_026_10')
    ]
)

# L0 Step 2
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT2 - L0PFS2',
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0S',
                'metadata_source': '.*RAW_025_10_.*'
            },
            {
                'file_type': f'{swath_id}_RAWP_0M',
                'metadata_source': '.*RAW_025_10_.*'
            }
        ]
    }
)

# L0 Step 3
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT2 - L0PFS3',
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',                
            }
        ]
    }
)

# L0 Step 4
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT2 - L0PFS4',
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(rcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

# L0 Step 6
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT2 - OAPF',
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': 'SM'
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

In [441]:
with open('int-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1a/b

In [442]:
with open('int-tds-l1-scenarios.json') as fh:
    config = json.load(fh)

In [443]:
config['scenarios'].extend(
    [
        {
            'name': f'INT_TDS - DT2 - L1PF - slice {slice_number}',
            'file_name': 'l1pf.sh',
            'processor_name': 'L1PF',
            'processor_version': '01.00',
            'task_name': 'L1PF',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': BASELINE,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'data_takes': [
                {
                    'data_take_id': str(dtid),
                    'start': data_take.left.isoformat() + 'Z',
                    'stop': data_take.right.isoformat() + 'Z',
                    'swath': swath_id,
                    'operational_mode': 'SM'
                }
            ],
            'mission_phase': mission_phase,
            'global_coverage_id': str(gcid),
            'major_cycle_id': str(gcid),
            'repeat_cycle_id': str(gcid),
            'track_nr': str(track),  # ISSUE: track_nr should be a list
            'footprint_polygon': footprint_polygon,
            'outputs': [
                {
                    'file_type': f'{swath_id}_SCS__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_SCS__1M',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_DGM__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                }
            ]
        }
        for slice_number in slices['n']
    ]
)

In [444]:
with open('int-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### MM - Stack - third data take

In [445]:
acq_idx = 553

In [446]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-08 22:53:13.688000', '2017-01-08 22:56:33.734000', closed='both')

In [447]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

931498

#### Mission phase, global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [448]:
mission_phase = 'INTERFEROMETRIC'
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [449]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [450]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-08 23:26:33.734000')

#### Affected Orbits

Orbits overlapping data take.

In [451]:
orbits = [
    pd.Interval(simulation.loc[acq_idx]['Anx'], simulation.loc[acq_idx]['NextAnx'], closed='both'),
    pd.Interval(
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['Anx'].min(),
        simulation[simulation['Orbit'] == simulation.loc[acq_idx]['Orbit'] + 1]['NextAnx'].min(),
        closed='both'
    )
]
orbits

[Interval('2017-01-08 21:16:31.200000', '2017-01-08 22:54:42.181000', closed='both'),
 Interval('2017-01-08 22:54:42.181000', '2017-01-09 00:32:53.163000', closed='both')]

#### Level 0 slices

Slice intervals overlapping data take:

In [452]:
slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    slices.extend(sg.intervals(data_take, 27))
    
slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in slices]
)
slices

,n,start,stop,duration
0,62,2017-01-08 22:53:13.688,2017-01-08 22:54:49.181,95.493
1,1,2017-01-08 22:54:37.181,2017-01-08 22:56:33.734,116.553


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [453]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = []
for orbit in orbits:
    sg = Grid(orbit.left, orbit.right, const.SLICE_GRID_DURATION, const.SLICE_INITIAL_OVERLAP, const.SLICE_FINAL_OVERLAP, const.NUM_SLICES)
    plt_slices.extend(sg.intervals(plt_data_validity, 27))
            
plt_slices = pd.DataFrame(
    [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
     for s in plt_slices]
)
plt_slices

,n,start,stop,duration
0,62,2017-01-08 22:52:57.688,2017-01-08 22:54:49.181,111.493
1,1,2017-01-08 22:54:37.181,2017-01-08 22:56:33.734,116.553


#### Level 1 frames

Frame intervals overlapping data take:

In [454]:
frames = []
for orbit in orbits:
    fg = Grid(orbit.left, orbit.right, const.FRAME_GRID_DURATION, const.FRAME_INITIAL_OVERLAP, const.FRAME_FINAL_OVERLAP, const.NUM_FRAMES)
    frames.extend(fg.intervals(data_take, 7))

frames = pd.DataFrame(
    [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
     for f in frames]
)
frames

,n,start,stop,duration
0,306,2017-01-08 22:53:13.688000000,2017-01-08 22:53:28.167433688,14.479433
1,307,2017-01-08 22:53:26.167433688,2017-01-08 22:53:47.170595236,21.003161
2,308,2017-01-08 22:53:45.170595236,2017-01-08 22:54:06.173756784,21.003161
3,309,2017-01-08 22:54:04.173756784,2017-01-08 22:54:25.176918332,21.003161
4,310,2017-01-08 22:54:23.176918332,2017-01-08 22:54:44.181000000,21.004081
5,1,2017-01-08 22:54:42.181000000,2017-01-08 22:55:03.184161548,21.003161
6,2,2017-01-08 22:55:01.184161548,2017-01-08 22:55:22.187323096,21.003161
7,3,2017-01-08 22:55:20.187323096,2017-01-08 22:55:41.190484644,21.003161
8,4,2017-01-08 22:55:39.190484644,2017-01-08 22:56:00.193646192,21.003161
9,5,2017-01-08 22:55:58.193646192,2017-01-08 22:56:19.196807740,21.003161


In [455]:
frame310_3 = pd.Interval(frames[frames['n'] == 310]['start'].iloc[0], frames[frames['n'] == 310]['stop'].iloc[0], closed='both')
frame310_3

Interval('2017-01-08 22:54:23.176918332', '2017-01-08 22:54:44.181000', closed='both')

#### Raw Data generation

Scenario generation

In [456]:
with open('int-tds-raw-gen-scenarios.json') as fh:
    config = json.load(fh)

In [457]:
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT3',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': acquisition_station,
        'baseline': BASELINE,
        'outputs': [
            {
                'file_type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'file_type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [25, 26]
        ]
    }
)

In [458]:
with open('int-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [459]:
with open('int-tds-l0-scenarios.json') as fh:
    config = json.load(fh)

In [460]:
# L0 Step 1
config['scenarios'].extend(
    [
        {
            'name': f'INT_TDS - DT3 - L0PFS1 - {file_type}',
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{file_type[5:7]}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{file_type[5:7]}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': BASELINE,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'file_type': file_type,
                    'metadata_source': f'.*{file_type}_.*'
                }
            ]
        }
        for file_type in ('RAW_022_10', 'RAW_023_10', 'RAW_024_10', 'RAW_025_10', 'RAW_026_10')
    ]
)

# L0 Step 2
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT3 - L0PFS2',
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0S',
                'metadata_source': '.*RAW_025_10_.*'
            },
            {
                'file_type': f'{swath_id}_RAWP_0M',
                'metadata_source': '.*RAW_025_10_.*'
            }
        ]
    }
)

# L0 Step 3
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT3 - L0PFS3',
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(mcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0M',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',                
            }
        ]
    }
)

# L0 Step 4
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT3 - L0PFS4',
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(rcid),
        'repeat_cycle_id': str(rcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

# L0 Step 6
config['scenarios'].append(
    {
        'name': f'INT_TDS - DT3 - OAPF',
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': 'SM'
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                'begin_position': data_take.left.isoformat() + 'Z',
                'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
)

In [461]:
with open('int-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1a/b

In [462]:
with open('int-tds-l1-scenarios.json') as fh:
    config = json.load(fh)

In [463]:
config['scenarios'].extend(
    [
        {
            'name': f'INT_TDS - DT3 - L1PF - slice {slice_number}',
            'file_name': 'l1pf.sh',
            'processor_name': 'L1PF',
            'processor_version': '01.00',
            'task_name': 'L1PF',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': BASELINE,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'data_takes': [
                {
                    'data_take_id': str(dtid),
                    'start': data_take.left.isoformat() + 'Z',
                    'stop': data_take.right.isoformat() + 'Z',
                    'swath': swath_id,
                    'operational_mode': 'SM'
                }
            ],
            'mission_phase': mission_phase,
            'global_coverage_id': str(gcid),
            'major_cycle_id': str(gcid),
            'repeat_cycle_id': str(gcid),
            'track_nr': str(track),  # ISSUE: track_nr should be a list
            'footprint_polygon': footprint_polygon,
            'outputs': [
                {
                    'file_type': f'{swath_id}_SCS__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_SCS__1M',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_DGM__1S',
                    'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                }
            ]
        }
        for slice_number in slices['n']
    ]
)

In [464]:
with open('int-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1c / level 2a

In [465]:
config = {
    'mission': 'biomass',
    'scenarios': []
}

In [466]:
config['scenarios'].append(
    {
        'name': f'INT_TDS - STACK',
        'file_name': 'stack_l2a.sh',
        'processor_name': 'STACK_L2A',
        'processor_version': '01.00',
        'task_name': 'STACK',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(gcid),
        'repeat_cycle_id': str(gcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'footprint_polygon': footprint_polygon,
        'outputs': [
            {
                'file_type': f'{swath_id}_STA__1S',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_1.right.isoformat() + 'Z'
            },
            {
                'file_type': f'{swath_id}_STA__1S',
                'begin_position': frame310_2.left.isoformat() + 'Z',
                'end_position': frame310_2.right.isoformat() + 'Z'
            },
            {
                'file_type': f'{swath_id}_STA__1S',
                'begin_position': frame310_3.left.isoformat() + 'Z',
                'end_position': frame310_3.right.isoformat() + 'Z'
            }
        ]
    }
)

config['scenarios'].append(
    {
        'name': f'INT_TDS - L2A',
        'file_name': 'stack_l2a.sh',
        'processor_name': 'STACK_L2A',
        'processor_version': '01.00',
        'task_name': 'L2A',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': BASELINE,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(dtid),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': 'SM'
            }
        ],
        'mission_phase': mission_phase,
        'global_coverage_id': str(gcid),
        'major_cycle_id': str(gcid),
        'repeat_cycle_id': str(gcid),
        'track_nr': str(track),  # ISSUE: track_nr should be a list
        'footprint_polygon': footprint_polygon,
        'outputs': [
            {
                'file_type': 'FP_FD__L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_3.right.isoformat() + 'Z'
            },
            {
                'file_type': 'FP_FH__L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_3.right.isoformat() + 'Z'
            },
            {
                'file_type': 'FP_ACM_L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_3.right.isoformat() + 'Z'
            },
            {
                'file_type': 'FP_VBG_L2A',
                'begin_position': frame310_1.left.isoformat() + 'Z',
                'end_position': frame310_3.right.isoformat() + 'Z'
            }
        ]
    }
)

In [467]:
with open('int-tds-l1c-l2a-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

## AUX Data generation

Scenario generation

In [5]:
baseline = 1

config = {
    'mission': 'biomass',
    'scenarios': [
        {
            'name': file_type,
            'file_name': 'N/A',
            'processor_name': 'procsim',
            'processor_version': procsim_version,
            'task_name': 'N/A',
            'task_version': 'N/A',
            'log_level': 'debug',
            'begin_position': '2000-01-01T00:00:00.000Z',
            'end_position': '2050-01-01T00:00:00.000Z',
            'baseline': baseline,
            'outputs': [
                {'type': file_type}
            ]
        }
        for file_type in ('AUX_INS___', 'AUX_PP0___', 'AUX_PP1___', 'AUX_PPS___', 'AUX_PP2A__')
    ]
}

# Processing Parameters
config['scenarios'].append(
    {
        'name': 'all',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': '2000-01-01T00:00:00.000Z',
        'end_position': '2050-01-01T00:00:00.000Z',
        'baseline': baseline,
        'outputs': [
            {'type': file_type}
            for file_type in ('AUX_INS___', 'AUX_PP0___', 'AUX_PP1___', 'AUX_PPS___', 'AUX_PP2A__')
        ]
    }
)

In [6]:
with open('aux-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)